$\newcommand{\To}{\Rightarrow}$
$\newcommand{\false}{\mathrm{false}}$

In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import BoolType
from kernel.term import Var, And, Or, Not, Inst
from kernel.thm import Thm
from kernel import theory
from logic import basic
from logic import matcher
from syntax.settings import settings

basic.load_theory('logic_base')
settings.unicode = True

## Propositional logic

In this section, we describe proofs in propositional logic. Propositional logic has four operators: conjunction ($\wedge$), disjunction ($\vee$), implication ($\to$), and negation ($\neg$). Rules for implication have already been discussed (primitive deduction rules `implies_intr` and `implies_elim`). We will now discuss rules for the other three operators in turn.

### Conjunction

The introduction and destruction rules for conjunction are `conjI`, `conjD1` and `conjD2`:

In [3]:
print('conjI: ', theory.get_theorem('conjI'))
print('conjD1:', theory.get_theorem('conjD1'))
print('conjD2:', theory.get_theorem('conjD2'))

conjI:  ⊢ ?A ⟶ ?B ⟶ ?A ∧ ?B
conjD1: ⊢ ?A ∧ ?B ⟶ ?A
conjD2: ⊢ ?A ∧ ?B ⟶ ?B


We consider these three theorems to be *axioms* that form the fundamental assumptions about conjunction (it is also possible to define conjunction satisfying these axioms in higher-order logic. This is omitted for simplicity). `conjI` states how a conjunction theorem can be proved. `conjD1` and `conjD2` shows how to make use of conjunction theorems.

We now give some examples for working with conjunction.

#### Example:

Prove $A \wedge B \to B \wedge A$.

#### Solution:

0. $A \wedge B \vdash A \wedge B$ by assume $A \wedge B$.
1. $\vdash A \wedge B \to A$ by theorem conjD1.
2. $A \wedge B \vdash A$ by implies_elim from 1, 0.
3. $\vdash A \wedge B \to B$ by theorem conjD2.
4. $A \wedge B \vdash B$ by implies_elim from 3, 0.
5. $\vdash A \to B \to A \wedge B$ by theorem conjI.
6. $\vdash B \to A \to B \wedge A$ by substitution {A: B, B: A} from 5.
7. $A \wedge B \vdash A \to B \wedge A$ by implies_elim 6, 4.
8. $A \wedge B \vdash B \wedge A$ by implies_elim 7, 2.
9. $\vdash A \wedge B \to B \wedge A$ by implies_intr $A \wedge B$ from 8.

In [4]:
A = Var("A", BoolType)
B = Var("B", BoolType)
th0 = Thm.assume(And(A, B))
th1 = Thm.substitution(Inst(A=A, B=B), theory.get_theorem('conjD1'))
th2 = Thm.implies_elim(th1, th0)
th3 = Thm.substitution(Inst(A=A, B=B), theory.get_theorem('conjD2'))
th4 = Thm.implies_elim(th3, th0)
th5 = theory.get_theorem('conjI')
th6 = Thm.substitution(Inst(A=B,B=A), th5)
th7 = Thm.implies_elim(th6, th4)
th8 = Thm.implies_elim(th7, th2)
th9 = Thm.implies_intr(And(A, B), th8)
print(th9)

⊢ A ∧ B ⟶ B ∧ A


### Interlude: automation for applying a theorem

In the proof above, a frequently occurring pattern is applying an existing theorem stated in terms of implication: the theorem is introduced, substitution is possibly made to bring the theorem into the required form, then one or more `implies_elim` is used to discharge the assumptions using earlier sequents. This repeating pattern calls for proof automation. The function `apply_theorem`, given below, takes the current theory, the name of an existing theorem, and a list of sequents that are supposed to discharge the assumptions of the theorem. It returns a sequent whose proposition is the conclusion of the theorem. It also takes an optional argument specifying instantiations of some of the variables (this will be useful in the section on disjunction).

In [5]:
def apply_theorem(th_name, *args, inst=None):
    th = theory.get_theorem(th_name)
    As, _ = th.prop.strip_implies()   # list of assumptions of th
    if inst is None:
        inst = Inst()                 # initial (empty) instantiation
    for A, arg in zip(As, args):      # match each assumption with corresponding arg
        inst = matcher.first_order_match(A, arg.prop, inst)
    th2 = Thm.substitution(inst, th)  # perform substitution on th
    for arg in args:                  # perform implies_elim on th
        th2 = Thm.implies_elim(th2, arg)
    return th2

The above code uses `strip_implies` function, which is very useful for extracting the assumption and conclusion of a theorem (note theorems stored in a theory usually have empty antecedent. Hence `th.prop` contains the full statement. For example:

In [6]:
th = theory.get_theorem('conjI')
print("prop:", th.prop)
As, C = th.prop.strip_implies()
print("As:", ", ".join(str(A) for A in As))
print("C: ", C)

prop: ?A ⟶ ?B ⟶ ?A ∧ ?B
As: ?A, ?B
C:  ?A ∧ ?B


Let's test the above function on each of `conjI`, `conjD1` and `conjD2` individually.

In [7]:
thA = Thm.assume(A)
thB = Thm.assume(B)
print(apply_theorem("conjI", thA, thB))

thAB = Thm.assume(And(A, B))
print(apply_theorem("conjD1", thAB))

print(apply_theorem("conjD2", thAB))

A, B ⊢ A ∧ B
A ∧ B ⊢ A
A ∧ B ⊢ B


Now we combine these to get a shorter (and clearer) proof of commutativity of conjunction:

In [8]:
th0 = Thm.assume(And(A, B))
th1 = apply_theorem('conjD1', th0)
th2 = apply_theorem('conjD2', th0)
th3 = apply_theorem('conjI', th2, th1)
th4 = Thm.implies_intr(And(A, B), th3)
print(th4)

⊢ A ∧ B ⟶ B ∧ A


In text, it can be written as follows:

0. $A \wedge B \vdash A \wedge B$ by assume $A \wedge B$.
1. $A \wedge B \vdash A$ by apply_theorem conjD1 from 0.
2. $A \wedge B \vdash B$ by apply_theorem conjD2 from 0.
3. $A \wedge B \vdash B \wedge A$ by apply_theorem conjI from 2, 1.
4. $\vdash A \wedge B \to B \wedge A$ by implies_intr $A \wedge B$ from 3.

### Disjunction

The introduction and elimination rules for disjunction are `disjI1`, `disjI2` and `disjE`:

In [9]:
print("disjI1:", theory.get_theorem('disjI1'))
print("disjI2:", theory.get_theorem('disjI2'))
print("disjE: ", theory.get_theorem('disjE'))

disjI1: ⊢ ?A ⟶ ?A ∨ ?B
disjI2: ⊢ ?B ⟶ ?A ∨ ?B
disjE:  ⊢ ?A ∨ ?B ⟶ (?A ⟶ ?C) ⟶ (?B ⟶ ?C) ⟶ ?C


Here, `disjI1` and `disjI2` are quite obvious, while `disjE` deserves some explanation. The `disjE` theorem says suppose we know $A \vee B$ and we want to prove some proposition $C$, it suffices to derive $C$ from $A$ (show $A \to C$) and to derive $C$ from $B$ (show $B \to C$). It corresponds to *case analysis* using the disjunction.

We demonstrate the rules for disjunction in the following example.

#### Example:

Prove $A \vee B \to B \vee A$.

#### Solution:

0. $A \vee B \vdash A \vee B$ by assume $A \vee B$.
1. $A \vdash A$ by assume $A$.
2. $A \vdash B \vee A$ by apply_theorem disjI2 {A: $B$} from 1.
3. $\vdash A \to B \vee A$ by implies_intr $A$ from 2.
4. $B \vdash B$ by assume $B$.
5. $B \vdash B \vee A$ by apply_theorem disjI1 {B: $A$} from 4.
6. $\vdash B \to B \vee A$ by implies_intr $B$ from 5.
7. $A \vee B \vdash B \vee A$ by apply_theorem disjE from 0, 3, 6.
8. $\vdash A \vee B \to B \vee A$ by implies_intr $A \vee B$ from 7.

Here, steps 1-3 is dedicated to proving $A \to B \vee A$, and steps 4-6 to proving $B \to B \vee A$, handling the two branches of case-analysis on $A \vee B$ with goal $B \vee A$. We can check this proof using the following Python code.

In [10]:
th0 = Thm.assume(Or(A, B))
th1 = Thm.assume(A)
th2 = apply_theorem('disjI2', th1, inst=Inst(A=B))
th3 = Thm.implies_intr(A, th2)
th4 = Thm.assume(B)
th5 = apply_theorem('disjI1', th4, inst=Inst(B=A))
th6 = Thm.implies_intr(B, th5)
th7 = apply_theorem('disjE', th0, th3, th6)
th8 = Thm.implies_intr(Or(A, B), th7)
print(th8)

⊢ A ∨ B ⟶ B ∨ A


### Negation

The negation operator is intimately linked to proof of contradiction ($\false$). In fact, it is usual to define $\neg A$ as $A \to \false$. This definition gives raise to the following two basic rules:

In [11]:
print("negI:", theory.get_theorem('negI'))
print("negE:", theory.get_theorem('negE'))

negI: ⊢ (?A ⟶ false) ⟶ ¬?A
negE: ⊢ ¬?A ⟶ ?A ⟶ false


The first rule states how to prove a negation: $\neg A$ can be proved by showing $A \to \false$. The second rule states how to use a negation: given $\neg A$, if in addition $A$ is known, then a contradiction can be derived.

#### Example:

Prove $A \to \neg \neg A$.

#### Solution:

0. $A \vdash A$ from assume $A$
1. $\neg A \vdash \neg A$ from assume $\neg A$.
2. $A, \neg A \vdash \false$ from apply_theorem negE from 1, 0.
3. $A \vdash \neg A \to \false$ from implies_intr $\neg A$.
4. $A \vdash \neg \neg A$ from apply_theorem negI from 3.
5. $\vdash A \to \neg \neg A$ from implies_intr $A$ from 4.

Let's try to understand the thought process behind this proof. First, it is clear that we need to assume $A$, and the goal is to show $\neg \neg A$. By `negI`, it suffices to show $\neg A \to \false$, and lines 1-3 is dedicated to this. For this purpose, it suffices to assume $\neg A$ and show $\false$. Since we also assumed $A$, the contradiction follows by `negE`. This proof can be checked as follows:

In [12]:
th0 = Thm.assume(A)
th1 = Thm.assume(Not(A))
th2 = apply_theorem('negE', th1, th0)
th3 = Thm.implies_intr(Not(A), th2)
th4 = apply_theorem('negI', th3)
th5 = Thm.implies_intr(A, th4)
print(th5)

⊢ A ⟶ ¬¬A


## Classical logic

We have now stated introduction and elimination/destruction rules for all four operations in propositional logic: conjunction, disjunction, implication, and negation. However, this is not yet the complete picture. While the above rules can be used to prove a lot of theorems, there are statements that are valid but cannot be proved. In order to be able to prove everything that is valid, we need to introduce two more rules. They are called the principle of explosion and the law of excluded middle.

The principle of explosion states that anything can be proved from a contradiction. It is given the name `falseE`:

In [13]:
print(theory.get_theorem('falseE'))

⊢ false ⟶ ?A


The law of excluded middle states that given any proposition $A$, either $A$ or $\neg A$ holds (there is nothing in the middle). It is also known as the *classical axiom*, as this is the axiom distinguishing *classical logic* from *intuitionistic logic*. It is given the name `classical`:

In [14]:
print(theory.get_theorem('classical'))

⊢ ?A ∨ ¬?A


We now demonstrate the use of these two theorems on a statement that we cannot prove before - the converse of the previous example.

#### Example:

Prove $\neg \neg A \to A$.

#### Solution:

0. $\neg \neg A \vdash \neg \neg A$ by assume $\neg \neg A$.
1. $\vdash A \vee \neg A$ by apply_theorem classical {A: A}.
2. $A \vdash A$ by assume $A$.
3. $\vdash A \to A$ by implies_intr $A$ from 2.
4. $\neg A \vdash \neg A$ by assume $\neg A$.
5. $\neg A, \neg \neg A \vdash \false$ by apply_theorem negE from 0, 4.
6. $\neg A, \neg \neg A \vdash A$ by apply_theorem falseE {A; A} from 5.
7. $\neg \neg A \vdash \neg A \to A$ by implies_intr $\neg A$ from 6.
8. $\neg \neg A \vdash A$ by apply_theorem disjE from 1, 3, 7.
9. $\vdash \neg \neg A \to A$ by implies_intr $\neg \neg A$ from 8.

This proof used both principle of explosion and law of excluded middle. First, law of excluded middle is used to create a case analysis on $A$ and $\neg A$. Lines 2-3 shows the $A$ case. Lines 4-7 shows the $\neg A$ case, during which the principle of explosion is used.

In [15]:
th0 = Thm.assume(Not(Not(A)))
th1 = apply_theorem('classical', inst=Inst(A=A))
th2 = Thm.assume(A)
th3 = Thm.implies_intr(A, th2)
th4 = Thm.assume(Not(A))
th5 = apply_theorem('negE', th0, th4)
th6 = apply_theorem('falseE', th5, inst=Inst(A=A))
th7 = Thm.implies_intr(Not(A), th6)
th8 = apply_theorem('disjE', th1, th3, th7)
th9 = Thm.implies_intr(Not(Not(A)), th8)
print(th9)

⊢ ¬¬A ⟶ A
